In [35]:
#Import your dependencies and read in the charity_data.csv to a Pandas DataFrame.
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
optimization_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
optimization_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


# Preprocess the dataset as you did in Step 1. Be sure to adjust for any modifications that came out of optimizing the model.

In [36]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
optimization_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [37]:
optimization_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [40]:
# Choose a cutoff value and create a list of application types to be replaced
cutoff_value_app = 500 
application_type_counts = optimization_df['APPLICATION_TYPE'].value_counts() 
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value_app].index.tolist() 
optimization_df['APPLICATION_TYPE'] = optimization_df['APPLICATION_TYPE'].apply(lambda x: 'Other' if x in application_types_to_replace else x)

# rare classifications
cutoff_value_cls = 1000
classification_counts = optimization_df['CLASSIFICATION'].value_counts() 
classifications_to_replace = classification_counts[classification_counts <= cutoff_value_cls].index.tolist()
optimization_df['CLASSIFICATION'] = optimization_df['CLASSIFICATION'].apply(lambda x: 'Other' if x in classifications_to_replace else x)

In [41]:
# Look at CLASSIFICATION value counts for binning
classification_counts = optimization_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [8]:
# scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(optimization_df.drop(columns=['IS_SUCCESSFUL']))
y = optimization_df['IS_SUCCESSFUL']
# datat split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y)

In [46]:
# Convert categorical data to numeric with `pd.get_dummies`
optimization_df_encoded = pd.get_dummies(optimization_df)

In [47]:
# Split our preprocessed data into our features and target arrays
# Split the data into features (X) and target (y)
X = optimization_df_encoded.drop(columns=['IS_SUCCESSFUL'])
y = optimization_df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [48]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [50]:
# Check the shapes of training and testing datasets
print("X_train shape:", X_train_scaled.shape)
print("X_test shape:", X_test_scaled.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (25724, 43)
X_test shape: (8575, 43)
y_train shape: (25724,)
y_test shape: (8575,)


## Compile, Train and Evaluate the Model

In [74]:
# Define the model - deep neural net
# hidden layer 3 was added for a better accuracy
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 150
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Additional hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model with a different learning rate
nn.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 150)               6600      
                                                                 
 dense_12 (Dense)            (None, 100)               15100     
                                                                 
 dense_13 (Dense)            (None, 50)                5050      
                                                                 
 dense_14 (Dense)            (None, 1)                 51        
                                                                 
Total params: 26801 (104.69 KB)
Trainable params: 26801 (104.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [75]:
# Compile the model
## "Adam" was beblow 75%
## "SGD" was below 75%
## "RMSprop" was below 75%
## because of this layers were ajusted and the models were run again

nn.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [107]:
# Train the model
## had to add more epchos
## added a validation_split=0.15
nn.fit(X_train_scaled, y_train, epochs=100, verbose=1, validation_split=0.15)

Epoch 1/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5526 - accuracy: 0.7335 - val_loss: 0.5510 - val_accuracy: 0.7327
Epoch 2/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5505 - accuracy: 0.7326 - val_loss: 0.5475 - val_accuracy: 0.7340
Epoch 3/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5479 - accuracy: 0.7325 - val_loss: 0.5485 - val_accuracy: 0.7317
Epoch 4/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 0.7347 - val_loss: 0.5452 - val_accuracy: 0.7346
Epoch 5/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5466 - accuracy: 0.7336 - val_loss: 0.5469 - val_accuracy: 0.7345
Epoch 6/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5451 - accuracy: 0.7346 - val_loss: 0.5439 - val_accuracy: 0.7354
Epoch 7/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5453 - accuracy: 0.7346 - val_loss: 0.5446 - val_accuracy: 0.7367

In [108]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Check if the accuracy meets the target
target_accuracy = 0.75
if model_accuracy >= target_accuracy:
    print("Congratulations! The model accuracy is 75% or higher.")
else:
    print("The model accuracy is below 75%.")

268/268 - 0s - loss: 0.5905 - accuracy: 0.7278 - 302ms/epoch - 1ms/step
The model accuracy is below 75%.


In [55]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

c:\Users\johan\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
